In [12]:
%pylab ipympl

Populating the interactive namespace from numpy and matplotlib


## Solution 

In [13]:
import astropy.units as u

In [14]:
Cd  = 0.3
rho = 1.225 *u.kg/u.m**3
A   = 1 * u.m**2
m = 80 * u.kg
k = 0.5*Cd*rho*A/m

In [15]:
print(k.to('1/m'))

0.002296875 1 / m


In [16]:
k = k.value
g = 9.8          # m/s^2 

In [17]:
global k,g, dt

In [18]:
h0   = 2000   # jump-off height [m]
h_p = 300    # end of free flight, pull parachute height [m]

We must integrate both the velocity as well as the distance according to 
$$
\frac{dh}{dt} = v
$$


In [19]:
def rhs_sdiver(x,dt):
    'Evaluate RHS for skydiver problem, advance one time step'
    h = x[0] + dt*x[1]
    v = x[1] + dt*(k*x[1]**2 - g)
    return [h,v]

We must also estimate a reasonable time step. 

In [20]:
dh = 10  # typical spatial step at terminal velocity [m]
v_t = sqrt(g/k)
dt = dh / v_t
print('%5.3f'%dt,'s')

0.153 s


In [21]:
hv_all = []
hv = array([h0,0])
while hv[0] > h_p:
    hv = rhs_sdiver(hv,dt)
    hv_all.append(hv)

In [22]:
hv_10=array(hv_all)

In [23]:
v_t

65.31972647421809

In [24]:
hv_plot_10=hv_10.T
close(1),figure(1)
plot(hv_plot_10[0],hv_plot_10[1],label='dh= 10m')
plot((hv_plot_10[0][0],hv_plot_10[0][-1]),(-v_t,-v_t),'k-')
xlabel('h / [m]')
ylabel('v / [km/s]')
legend(loc=1)
xlim(2100,250)

FigureCanvasNbAgg()

(2100, 250)

## Convergence test

In [25]:
t_max = 15

dh = [10,100, 300]
close(2)
figure(2)
for this_dh in dh:
    dt = this_dh / v_t
    tv_all = [[0,0]] # each element is [t, v]
    hv = array([0,0]); t = 0
    while t < t_max:
        hv = rhs_sdiver(hv,dt)
        t += dt
        tv_all.append([t,hv[1]])
    tv = array(tv_all).T
    plot(tv[0],tv[1],'.-',label='dh='+str(this_dh)+'m')
xlabel('t / [s]')
ylabel('v / [m/s]')
legend(loc=1)


FigureCanvasNbAgg()

In [26]:
close(2);figure(2)
dh_range = [5,10,20,40,80,160]
#dh_range = [2,100]
for dh in dh_range[::-1]:
    dt = dh / v_t
    print('time step %5.3f'%dt,'s')
    hv_all = []
    hv = array([2000,0])
    while hv[0] > h_p:
        hv = rhs_sdiver(hv,dt)
        hv_all.append(hv)
    hv_plot = array(hv_all).T    
    plot(hv_plot[0],log10(hv_plot[1]+v_t),label='dh='+str(dh)+'m')
xlabel('h [m]')
ylabel('$ \log (v-v_\mathrm{t}) \mathrm{\ [km/s]}$')
legend(loc=1)
xlim(2100,250)

FigureCanvasNbAgg()

time step 2.449 s
time step 1.225 s
time step 0.612 s
time step 0.306 s
time step 0.153 s
time step 0.077 s


(2100, 250)

## Solution for system with odeint
Plot H and v as a function of t.

In [47]:
def rhs_sdiver_odeint(y,t,k,g):
    'Evaluate RHS for skydiver problem, advance one time step'
    hrhs = y[1]
    vrhs = k*y[1]**2 - g
    return [hrhs,vrhs]


In [48]:
from scipy import integrate

In [49]:
t=linspace(0,25,25)
y = integrate.odeint(rhs_sdiver_odeint,[h0,0],t, args=(k,g))

In [50]:
close(5);figure(5)
plot(t,y.T[0])
ylabel('height / m')
xlabel('time / s')

FigureCanvasNbAgg()

Text(0.5, 0, 'time / s')

In [51]:
pwd


'/home/user/repos/mp248-planning/Labs'

In [52]:
close(6);figure(6)
plot(t,y.T[1])
ylabel('velocity / m/s')
xlabel('time / s')

FigureCanvasNbAgg()

Text(0.5, 0, 'time / s')